In this tutorial, we'll learn how to call run_compute to create a synthetic model for the attached datasets.

# Setup

In [1]:
import phoebe
from phoebe import u,c

import numpy as np
import matplotlib.pyplot as plt

/usr/lib/python2.7/dist-packages/matplotlib/cbook/deprecation.py:106: MatplotlibDeprecationWarning: The mpl_toolkits.axes_grid module was deprecated in version 2.1. Use mpl_toolkits.axes_grid1 and mpl_toolkits.axisartist provies the same functionality instead.
  warnings.warn(message, mplDeprecation, stacklevel=1)


In [2]:
logger = phoebe.logger(clevel='WARNING')

In [3]:
b = phoebe.default_binary()

# Compute

Let's first re-add some simple datasets so that we can create the models.

In [4]:
b.add_dataset('lc', times=np.linspace(0,1,51), dataset='lc01')

<ParameterSet: 15 parameters | contexts: compute, dataset>

In [5]:
b.add_dataset('rv', times=np.linspace(0,1,21), dataset='rv01')

<ParameterSet: 15 parameters | contexts: compute, dataset>

Now that we have datasets, we could compute the forward model with all the defaults by calling run_compute

In [6]:
b.run_compute()

<ParameterSet: 6 parameters | kinds: rv, lc>

This creates synthetic versions of each dataset copied into the with context='model'.  Let's filter to see what we can find in the model.

In [7]:
b.filter(context='model').datasets

['lc01', 'rv01']

In [8]:
b.filter(context='model', dataset='lc01')

<ParameterSet: 2 parameters | qualifiers: fluxes, times>

For light curves, the synthetic model consists of the times (copied directly from the values provided in the dataset) and the synthetic fluxes.

In [9]:
b.get_parameter(context='model', dataset='lc01', qualifier='times')

<Parameter: times=[0.   0.02 0.04 ... 0.96 0.98 1.  ] d | keys: description, value, default_unit, visible_if, copy_for>

In [10]:
b.get_parameter(context='model', dataset='lc01', qualifier='fluxes')

<Parameter: fluxes=[0.98283437 1.3642039  1.76397113 ... 1.76375009
 1.36402934 0.98283437] W / m2 | keys: description, value, default_unit, visible_if, copy_for>

For radial velocities, the synthetic model consists again of the times, as well as the RVs for both stars in the system.  Therefore, we also need to provide the component tag in order to access a single Parameter.

In [11]:
b.filter(context='model', dataset='rv01').qualifiers

['rvs', 'times']

In [12]:
b.get_parameter(context='model', datset='rv01', component='primary', qualifier='rvs')

<Parameter: rvs=[  2.22768178 -44.86212329 -78.66881743 ...  78.6648444
  44.8532587    2.22768178] km / s | keys: description, value, default_unit, visible_if, copy_for>

In [13]:
b.get_parameter(context='model', datset='rv01', component='secondary', qualifier='rvs')

<Parameter: rvs=[-1.42330848e-04  4.14275175e+01  7.87825229e+01 ...
 -7.87801299e+01 -4.14257699e+01 -1.42330848e-04] km / s | keys: description, value, default_unit, visible_if, copy_for>

### Custom Compute Options

As we mentioned earlier when adding datasets, Parameters with context='compute' control options for how the model is computed.  With a default Bundle, there is already a single set of compute options attached (these were what was used when we called run_compute above).

In [14]:
print b.filter(context='compute').qualifiers

['dynamics_method', 'pbmesh', 'boosting_method', 'horizon_method', 'lc_method', 'atm', 'enabled', 'ltte', 'rv_grav', 'rv_method', 'eclipse_method', 'ntriangles', 'horizon', 'fti_method', 'distortion_method', 'mesh_method', 'protomesh', 'irrad_method']


These are options that tell the backend which methods to use or which advanced effects to consider.  Because of this, these options can be changed to either create a quick-and-dirty fast model, or a robust-but-slow model.

Similar to datasets, we can add additional sets of compute options.  So let's create one set of compute options for quick-and-dirty computations (and tag it compute='preview')

In [15]:
b.add_compute(compute='preview')

<ParameterSet: 25 parameters | datasets: lc01, rv01>

Now if we filter for any of the Parameters with context='compute', we'll see that there is a copy for each of our created set of compute options as well as the original default set.

In [16]:
b.filter(context='compute', qualifier='ltte')

<ParameterSet: 2 parameters | computes: phoebe01, preview>

Now let's set a few options to turn off some effects. Let's first disable all the advanced physics (ltte, rv_grav, irrad_method).

In [17]:
b.get_parameter(context='compute', compute='preview', qualifier='ltte').set_value(False)

If we try to do the same as above, but use qualifier='rv_grav', we'll get an error saying there are 2 results found.  This is because 'rv_grav' can be enabled/disabled for each individual star.  We can see this is the case by doing a filter.

In [18]:
b.filter(context='compute', compute='preview', qualifier='rv_grav')

<ParameterSet: 2 parameters | components: primary, secondary>

We could either set the value for component='primary' and component='secondary' separately, or we can set them both at once by using set_value_all (this sets the value for all Parameters in a ParameterSet)

NOTE: this requires sending value as a keyword instead of an argument since set_value_all also accepts filter options

In [19]:
b.filter(context='compute', compute='preview', qualifier='rv_grav').set_value_all(value=False)

In [20]:
b.get_parameter(context='compute', compute='preview', qualifier='irrad_method').choices

['none', 'wilson', 'horvat']

In [21]:
b.get_parameter(context='compute', compute='preview', qualifier='irrad_method').set_value('none')

And let's also decrease the number of triangles used in the meshes.  Again, we can use set_value_all since there is an entry for each star.

In [22]:
b.filter(context='compute', compute='preview', qualifier='ntriangles').set_value_all(value=800)

Now if we want to call run_compute again, we **must** also pass which set of compute options we want to use (the default 'phoebe01', or our new 'preview' options)

In [23]:
b.run_compute(compute='preview')

Wed, 23 May 2018 13:25 BUNDLE       WARNING overwriting model: latest


<ParameterSet: 6 parameters | kinds: rv, lc>

### Temporarily Overriding Options

Any of these compute options can be overridden as keyword arguments to run_compute.  These won't change the Parameter values, and will only apply to that single call of run_compute.

In [24]:
b.run_compute(compute='preview', irrad_method='horvat')

Wed, 23 May 2018 13:26 BUNDLE       WARNING overwriting model: latest


<ParameterSet: 6 parameters | kinds: rv, lc>

In addition to the compute options, you can also override the times across **all** datasets

In [25]:
b.run_compute(compute='preview', times=[0, 0.5, 1])

Wed, 23 May 2018 13:26 BUNDLE       WARNING overwriting model: latest


<ParameterSet: 6 parameters | kinds: rv, lc>

In [26]:
print "dataset times: ", b.get_value('times', dataset='lc01', context='dataset')
print "model times: ", b.get_value('times', dataset='lc01', context='model')

dataset times:  [0.   0.02 0.04 0.06 0.08 0.1  0.12 0.14 0.16 0.18 0.2  0.22 0.24 0.26
 0.28 0.3  0.32 0.34 0.36 0.38 0.4  0.42 0.44 0.46 0.48 0.5  0.52 0.54
 0.56 0.58 0.6  0.62 0.64 0.66 0.68 0.7  0.72 0.74 0.76 0.78 0.8  0.82
 0.84 0.86 0.88 0.9  0.92 0.94 0.96 0.98 1.  ]
model times:  [0.  0.5 1. ]


### Alternate Backends

In addition to creating separate compute options for robust vs preview, compute options also serve to run backends other than phoebe2.  Currently there are experimental wrappers for PHOEBE 1.0 (legacy), jktebop, and photodynam.

Similar to providing 'lc' or 'rv' to add_dataset, you just need to provide the name of the backend to add_compute.

NOTE: calling run_compute with one of these alternate backends will fail unless you have the necessary code installed on your machine.

In [27]:
b.add_compute('legacy', compute='legacycompute')

<ParameterSet: 12 parameters | datasets: lc01, rv01>

In [28]:
print b.filter(compute='legacycompute')

ParameterSet: 12 parameters
  protomesh@legacycompute@com...: False
    pbmesh@legacycompute@compute: False
  refl_num@legacycompute@compute: 1
        ie@legacycompute@compute: False
  enabled@lc01@legacycompute@...: True
  enabled@rv01@legacycompute@...: True
  atm@primary@legacycompute@c...: extern_atmx
  atm@secondary@legacycompute...: extern_atmx
  gridsize@primary@legacycomp...: 60
  gridsize@secondary@legacyco...: 60
  rv_method@primary@legacycom...: flux-weighted
  rv_method@secondary@legacyc...: flux-weighted


# Exercise

Add another set of compute options (called, say, 'robust') and set whatever values you think might be necessary.  Run a model and see how much longer it takes to run.